In [1]:
import pandas as pd 
import numpy as np

In [2]:
orig_train = pd.read_csv('train.csv')
orig_test = pd.read_csv('test.csv')

In [3]:
partial_clean_train = pd.read_csv('stat154_cleaned_ryan.csv')
partial_clean_test = pd.read_csv('stat154_updated_test_ryan')

In [4]:
orig_train['Start_Time'] = pd.to_datetime(orig_train['Start_Time'])
orig_test['Start_Time'] = pd.to_datetime(orig_test['Start_Time'])

In [5]:
na_rows_train = np.where(orig_train['Sunrise_Sunset'].isnull())[0]
na_rows_test = np.where(orig_test['Sunrise_Sunset'].isnull())[0]

In [6]:
def fill_na_sunrise_sunset_train(row_idx, tbl=orig_train):
    state = tbl.loc[row_idx, 'State']
    month = tbl.loc[row_idx, 'Start_Time'].month
    hr = tbl.loc[row_idx, 'Start_Time'].hour
    filtered_rows = tbl[(tbl['State'] == state) & 
                       (tbl['Start_Time'].dt.month == month) &
                       (tbl['Start_Time'].dt.hour == hr)]
    replace = pd.Series.mode(filtered_rows['Sunrise_Sunset']).iloc[0]
    return replace

def fill_na_sunrise_sunset_test(row_idx, tbl=orig_test):
    state = tbl.loc[row_idx, 'State']
    month = tbl.loc[row_idx, 'Start_Time'].month
    hr = tbl.loc[row_idx, 'Start_Time'].hour
    filtered_rows = tbl[(tbl['State'] == state) & 
                       (tbl['Start_Time'].dt.month == month) &
                       (tbl['Start_Time'].dt.hour == hr)]
    replace = pd.Series.mode(filtered_rows['Sunrise_Sunset']).iloc[0]
    return replace

fill_na_map_train = np.vectorize(fill_na_sunrise_sunset_train)
fill_na_map_test = np.vectorize(fill_na_sunrise_sunset_test)

result_train = fill_na_map_train(na_rows_train)
result_test = fill_na_map_test(na_rows_test)

for i in range(len(result_train)):
    orig_train.loc[na_rows_train[i], 'Sunrise_Sunset'] = result_train[i]
    
for j in range(len(result_test)):
    orig_test.loc[na_rows_test[j], 'Sunrise_Sunset'] = result_test[j]

print(len(orig_train['Sunrise_Sunset'].dropna())/len(orig_train['Sunrise_Sunset']))
print(len(orig_test['Sunrise_Sunset'].dropna())/len(orig_test['Sunrise_Sunset']))

1.0
1.0


In [7]:
# Cleaning
partial_clean_train['Timestamp'] = pd.to_datetime(partial_clean_train['weather.ymd'])
partial_clean_train['Amenity'] = orig_train['Amenity']
partial_clean_train['Bump'] = orig_train['Bump']
partial_clean_train['Crossing'] = orig_train['Crossing']
partial_clean_train['Give_Way'] = orig_train['Give_Way']
partial_clean_train['Junction'] = orig_train['Junction']
partial_clean_train['No_Exit'] = orig_train['No_Exit']
partial_clean_train['Railway'] = orig_train['Railway']
partial_clean_train['Roundabout'] = orig_train['Roundabout']
partial_clean_train['Station'] = orig_train['Station']
partial_clean_train['Stop'] = orig_train['Stop']
partial_clean_train['Traffic_Calming'] = orig_train['Traffic_Calming']
partial_clean_train['Traffic_Signal'] = orig_train['Traffic_Signal']
partial_clean_train['Turning_Loop'] = orig_train['Turning_Loop']
partial_clean_train['Sunrise_Sunset'] = orig_train['Sunrise_Sunset']
partial_clean_train['Source'] = orig_train['Source']
partial_clean_train['Severity'] = orig_train['Severity']

In [8]:
partial_clean_test['Timestamp'] = pd.to_datetime(partial_clean_test['weather.ymd'])
partial_clean_test['Amenity'] = orig_test['Amenity']
partial_clean_test['Bump'] = orig_test['Bump']
partial_clean_test['Crossing'] = orig_test['Crossing']
partial_clean_test['Give_Way'] = orig_test['Give_Way']
partial_clean_test['Junction'] = orig_test['Junction']
partial_clean_test['No_Exit'] = orig_test['No_Exit']
partial_clean_test['Railway'] = orig_test['Railway']
partial_clean_test['Roundabout'] = orig_test['Roundabout']
partial_clean_test['Station'] = orig_test['Station']
partial_clean_test['Stop'] = orig_test['Stop']
partial_clean_test['Traffic_Calming'] = orig_test['Traffic_Calming']
partial_clean_test['Traffic_Signal'] = orig_test['Traffic_Signal']
partial_clean_test['Turning_Loop'] = orig_test['Turning_Loop']
partial_clean_test['Sunrise_Sunset'] = orig_test['Sunrise_Sunset']
partial_clean_test['Source'] = orig_test['Source']

In [ ]:
final_train = partial_clean_train.drop(columns=['Unnamed: 0', 'X','weather.ymd',
                                                        'weather.md',
                                                'weather.mon']).dropna()
def map_severity(severity):
    if severity == 1 or severity == 2:
        return 0
    elif severity == 3 or severity == 4:
        return 1
    
mapper = np.vectorize(map_severity)

final_train['Severity'] = mapper(final_train['Severity'].values)
final_train.head()

In [9]:
final_test = partial_clean_test.drop(columns=['Unnamed: 0', 'X', 'ID', 'weather.ymd',
                                              'weathertime', 'weather.md', 'weather.mon',
                                             'weather.hr'])
final_test = final_test.replace(np.nan, '', regex=True)
final_test.isnull().mean() * 100

temp.imp.zip          0.0
humid.imp.zip         0.0
pressure.imp.zip      0.0
visibility.imp.zip    0.0
wind.d.imp.zip        0.0
wind.speed.imp.zip    0.0
light                 0.0
heavy                 0.0
rain                  0.0
snow                  0.0
storm                 0.0
windy                 0.0
haze                  0.0
Start_Time            0.0
End_Time              0.0
Duration              0.0
Start_Lat             0.0
Start_Lng             0.0
Distance.mi           0.0
Description           0.0
Side                  0.0
City                  0.0
County                0.0
State                 0.0
Zipcode               0.0
Country               0.0
Timezone              0.0
Airport_Code          0.0
Weather_Timestamp     0.0
Timestamp             0.0
Amenity               0.0
Bump                  0.0
Crossing              0.0
Give_Way              0.0
Junction              0.0
No_Exit               0.0
Railway               0.0
Roundabout            0.0
Station     

In [ ]:
from sklearn.model_selection import train_test_split

train_complete, val_complete = train_test_split(final_train,
                                                test_size=0.2, random_state=1)

In [ ]:
train_complete.shape

In [ ]:
val_complete.shape

In [ ]:
val_complete.head()

In [ ]:
train_complete.to_csv('train_set.csv', index=False)
val_complete.to_csv('val_set.csv', index=False)

In [10]:
final_test.to_csv('test_set.csv', index=False)